In [14]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

In [15]:
def separate_city_team_nhl(row):
  composite_team_name = ['Maple Leafs', 'Red Wings', 'Blue Jackets', 'Golden Knights']

  if any(team_name in row['team'] for team_name in composite_team_name):
      row['team'] = row['team'].split(" ")[-2] + " " + row['team'].split(" ")[-1] 
  else:
      row['team'] = row['team'].split(" ")[-1]

  return row


def separate_city_team_nba(row):
  composite_team_name = ['Trail Blazers']

  if any(team_name in row['team'] for team_name in composite_team_name):
      row['team'] = row['team'].split(" ")[-2] + " " + row['team'].split(" ")[-1] 
  else:
      row['team'] = row['team'].split(" ")[-1]

  return row


def separate_city_team_mlb(row):
  composite_team_name = ['Red Sox', 'Blue Jays', 'White Sox']

  if any(team_name in row['team'] for team_name in composite_team_name):
      row['team'] = row['team'].split(" ")[-2] + " " + row['team'].split(" ")[-1] 
  else:
      row['team'] = row['team'].split(" ")[-1]

  return row

def separate_city_team_nfl(row):
  row['team'] = row['team'].split(" ")[-1]

  return row


def group_win_loss_ratio(group):
  avg_ratio = np.nanmean(group['Win ratio'])

  group['Win ratio'] = avg_ratio
  group['Population'] = group['Population']

  return group

In [16]:
def clean_nhl_df():
  divisions = ['Atlantic Division', 'Metropolitan Division', 'Central Division', 'Pacific Division']
  nhl_df = pd.read_csv("/content/nhl.csv")
  nhl_df = nhl_df[ pd.to_datetime(nhl_df['year'], format='%Y') == '2018']
  nhl_df = nhl_df[ ~nhl_df['team'].isin(divisions) ]
  nhl_df = nhl_df[['team', 'L', 'W']]
  nhl_df['team'] = nhl_df['team'].replace(to_replace='\*', value='', regex=True)
  nhl_df = nhl_df.apply(separate_city_team_nhl, axis=1)
  nhl_df = nhl_df.reset_index(drop=True)

  cities = pd.read_html("/content/wikipedia_data.html")[1]
  cities = cities.iloc[:-1,[0,3,5,6,7,8]]
  cities = cities.replace(to_replace=['—', '\[.*\]'], value='', regex=True)
  cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population'})
  cities = cities[['Metropolitan area', 'Population', 'NHL']]
  cities = cities[cities['NHL'] != '']

  multivalued_team_name = ['Rangers Islanders Devils', 'Kings Ducks']

  cities['NHL'][cities['NHL'].isin(multivalued_team_name)] = cities['NHL'][cities['NHL'].isin(multivalued_team_name)].str.split(" ")
  cities = cities.explode('NHL')
  cities = cities.reset_index(drop=True)

  cities_teams_nhl = pd.merge(cities, nhl_df, how='inner', left_on='NHL', right_on='team')
  cities_teams_nhl = cities_teams_nhl.rename(columns={'L': 'Loss', 'W': 'Wins'})
  cities_teams_nhl = cities_teams_nhl.drop(['NHL', 'team'], axis=1)
  cities_teams_nhl[['Population', 'Loss', 'Wins']] = cities_teams_nhl[['Population', 'Loss', 'Wins']].apply(pd.to_numeric) 
  cities_teams_nhl['Win ratio'] = cities_teams_nhl['Wins'] / (cities_teams_nhl['Wins'] + cities_teams_nhl['Loss'])
  cities_teams_nhl = cities_teams_nhl.groupby('Metropolitan area').apply(group_win_loss_ratio)
  cities_teams_nhl = cities_teams_nhl.drop_duplicates(subset='Metropolitan area', keep='first')
  cities_teams_nhl = cities_teams_nhl.sort_values('Metropolitan area')
  cities_teams_nhl = cities_teams_nhl.reset_index(drop=True)

  return cities_teams_nhl

In [17]:
def clean_nba_df():
  cities = pd.read_html("/content/wikipedia_data.html")[1]
  cities = cities.iloc[:-1,[0,3,5,6,7,8]]
  cities = cities.replace(to_replace=['—', '\[.*\]'], value='', regex=True)
  cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population'})
  cities = cities[['Metropolitan area', 'Population', 'NBA']]
  cities = cities[cities['NBA'] != '']

  multivalued_team_name = ['Knicks Nets', 'Lakers Clippers']

  cities['NBA'][cities['NBA'].isin(multivalued_team_name)] = cities['NBA'][cities['NBA'].isin(multivalued_team_name)].str.split(" ")
  cities = cities.explode('NBA')
  cities = cities.reset_index(drop=True)

  nba_df = pd.read_csv("/content/nba.csv")
  nba_df = nba_df[ pd.to_datetime(nba_df['year'], format='%Y') == '2018']
  nba_df = nba_df[['team', 'W', 'L']]
  nba_df['team'] = nba_df['team'].replace(to_replace=['\*', '\(\d+\)'], value='', regex=True)
  nba_df['team'] = nba_df['team'].apply(lambda x: x.strip())
  nba_df = nba_df.apply(separate_city_team_nba, axis=1)
  nba_df = nba_df.reset_index(drop=True)

  cities_teams_nba = pd.merge(cities, nba_df, how='inner', left_on='NBA', right_on='team')
  cities_teams_nba = cities_teams_nba.rename(columns={'L': 'Loss', 'W': 'Wins'})
  cities_teams_nba = cities_teams_nba.drop(['NBA', 'team'], axis=1)
  cities_teams_nba[['Population', 'Loss', 'Wins']] = cities_teams_nba[['Population', 'Loss', 'Wins']].apply(pd.to_numeric) 
  cities_teams_nba['Win ratio'] = cities_teams_nba['Wins'] / (cities_teams_nba['Wins'] + cities_teams_nba['Loss'])
  cities_teams_nba = cities_teams_nba.groupby('Metropolitan area').apply(group_win_loss_ratio)
  cities_teams_nba = cities_teams_nba.drop_duplicates(subset='Metropolitan area', keep='first')
  cities_teams_nba = cities_teams_nba.sort_values('Metropolitan area')
  cities_teams_nba = cities_teams_nba.reset_index(drop=True)

  return cities_teams_nba

In [18]:
def clean_mlb_df():
  mlb_df=pd.read_csv("/content/mlb.csv")
  mlb_df = mlb_df[ pd.to_datetime(mlb_df['year'], format='%Y') == '2018']
  mlb_df = mlb_df[['team', 'W', 'L']]
  mlb_df['team'] = mlb_df['team'].apply(lambda x: x.strip())
  mlb_df = mlb_df.apply(separate_city_team_mlb, axis=1)
  mlb_df = mlb_df.reset_index(drop=True)

  cities = pd.read_html("/content/wikipedia_data.html")[1]
  cities = cities.iloc[:-1,[0,3,5,6,7,8]]
  cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population'})
  cities = cities[['Metropolitan area', 'Population', 'MLB']]
  cities = cities.replace(to_replace=['—', '\[.*\]'], value='', regex=True)
  cities = cities[cities['MLB'] != '']

  multivalued_team_name = ['Yankees Mets', 'Dodgers Angels', 'Giants Athletics', 'Cubs White Sox']

  cities['MLB'][cities['MLB'].isin(multivalued_team_name)] = cities['MLB'][cities['MLB'].isin(multivalued_team_name)].str.split(" ")
  cities['MLB'][3] = ['Cubs', 'White Sox']
  cities = cities.explode('MLB')
  cities['MLB'] = cities['MLB'].apply(lambda x: x.strip())
  cities = cities.reset_index(drop=True)

  cities_teams_mlb = pd.merge(cities, mlb_df, how='inner', left_on='MLB', right_on='team')
  cities_teams_mlb = cities_teams_mlb.rename(columns={'L': 'Loss', 'W': 'Wins'})
  cities_teams_mlb = cities_teams_mlb.drop(['MLB', 'team'], axis=1)
  cities_teams_mlb[['Population', 'Loss', 'Wins']] = cities_teams_mlb[['Population', 'Loss', 'Wins']].apply(pd.to_numeric) 
  cities_teams_mlb['Win ratio'] = cities_teams_mlb['Wins'] / (cities_teams_mlb['Wins'] + cities_teams_mlb['Loss'])
  cities_teams_mlb = cities_teams_mlb.groupby('Metropolitan area').apply(group_win_loss_ratio)
  cities_teams_mlb = cities_teams_mlb.drop_duplicates(subset='Metropolitan area', keep='first')
  cities_teams_mlb = cities_teams_mlb.sort_values('Metropolitan area')
  cities_teams_mlb = cities_teams_mlb.reset_index(drop=True)

  return cities_teams_mlb

In [19]:
def clean_nfl_df():
  divisions = ['AFC East', 'AFC North', 'AFC South', 'AFC West', 'NFC East', 'NFC North', 'NFC South', 'NFC West']
  nfl_df = pd.read_csv("/content/nfl.csv")
  nfl_df = nfl_df[ pd.to_datetime(nfl_df['year'], format='%Y') == '2018']
  nfl_df = nfl_df[ ~nfl_df['team'].isin(divisions) ]
  nfl_df = nfl_df[['team', 'W', 'L']]
  nfl_df['team'] = nfl_df['team'].apply(lambda x: x.strip())
  nfl_df['team'] = nfl_df['team'].replace(to_replace=['\*', '\+'], value='', regex=True)
  nfl_df = nfl_df.apply(separate_city_team_nfl, axis=1)
  nfl_df = nfl_df.reset_index(drop=True)


  multivalued_team_name = ['Giants Jets', 'Rams Chargers', '49ers Raiders']
  cities=pd.read_html("/content/wikipedia_data.html")[1]
  cities=cities.iloc[:-1,[0,3,5,6,7,8]]
  cities = cities.rename(columns={'Population (2016 est.)[8]': 'Population'})
  cities = cities[['Metropolitan area', 'Population', 'NFL']]
  cities = cities.replace(to_replace=['—', '\[.*\]'], value='', regex=True)
  cities['NFL'] = cities['NFL'].apply(lambda x: x.strip())
  cities = cities[cities['NFL'] != '']
  cities['NFL'][cities['NFL'].isin(multivalued_team_name)] = cities['NFL'][cities['NFL'].isin(multivalued_team_name)].str.split(" ")
  cities = cities.explode('NFL')
  cities = cities.reset_index(drop=True)

  cities_teams_nfl = pd.merge(cities, nfl_df, how='inner', left_on='NFL', right_on='team')
  cities_teams_nfl = cities_teams_nfl.rename(columns={'L': 'Loss', 'W': 'Wins'})
  cities_teams_nfl = cities_teams_nfl.drop(['NFL', 'team'], axis=1)
  cities_teams_nfl[['Population', 'Loss', 'Wins']] = cities_teams_nfl[['Population', 'Loss', 'Wins']].apply(pd.to_numeric) 
  cities_teams_nfl['Win ratio'] = cities_teams_nfl['Wins'] / (cities_teams_nfl['Wins'] + cities_teams_nfl['Loss'])
  cities_teams_nfl = cities_teams_nfl.groupby('Metropolitan area').apply(group_win_loss_ratio)
  cities_teams_nfl = cities_teams_nfl.drop_duplicates(subset='Metropolitan area', keep='first')
  cities_teams_nfl = cities_teams_nfl.sort_values('Metropolitan area')
  cities_teams_nfl = cities_teams_nfl.reset_index(drop=True)

  return cities_teams_nfl

In [20]:
def nhl_correlation():
  cities_teams_nhl = clean_nhl_df()

  population_by_region = list(cities_teams_nhl['Population']) 
  win_loss_by_region = list(cities_teams_nhl['Win ratio']) 

  return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [21]:
def nba_correlation():
  cities_teams_nba = clean_nba_df()

  population_by_region = list(cities_teams_nba['Population']) 
  win_loss_by_region = list(cities_teams_nba['Win ratio']) 

  return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [22]:
def mlb_correlation():
  cities_teams_mlb = clean_mlb_df()

  population_by_region = list(cities_teams_mlb['Population']) 
  win_loss_by_region = list(cities_teams_mlb['Win ratio']) 

  return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [23]:
def nfl_correlation():
  cities_teams_nfl = clean_nfl_df()

  population_by_region = list(cities_teams_nfl['Population']) 
  win_loss_by_region = list(cities_teams_nfl['Win ratio']) 

  return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [24]:
nhl_correlation()

<ipython-input-16-6004683283bc>:29: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  cities_teams_nhl = cities_teams_nhl.groupby('Metropolitan area').apply(group_win_loss_ratio)


0.012486162921209923

In [25]:
nba_correlation()

<ipython-input-17-0cb7acdadc4a>:28: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  cities_teams_nba = cities_teams_nba.groupby('Metropolitan area').apply(group_win_loss_ratio)


-0.17657160252844614

In [26]:
mlb_correlation()

<ipython-input-18-9c96ced8dfd3>:29: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  cities_teams_mlb = cities_teams_mlb.groupby('Metropolitan area').apply(group_win_loss_ratio)


0.15027698302669307

In [27]:
nfl_correlation()

<ipython-input-19-c6c3f8d579a2>:30: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  cities_teams_nfl = cities_teams_nfl.groupby('Metropolitan area').apply(group_win_loss_ratio)


0.004922112149349409